In [1]:
from utils import *

hete = Heterogeneity()
hete.check_torch_gpu()

-------------------------------------------------
------------------ VERSION INFO -----------------
Conda Environment: torchy | Python version: 3.8.16 (default, Mar  2 2023, 03:18:16) [MSC v.1916 64 bit (AMD64)]
Torch version: 2.0.1
Torch build with CUDA? True
# Device(s) available: 1, Name(s): Quadro P520



In [2]:
n_realizations = 5
n_timesteps    = 61
dim            = 256

pressure, sat_h2 = np.zeros((n_realizations,n_timesteps,dim,dim,1)), np.zeros((n_realizations,n_timesteps,dim,dim,1))
poro,     perm   = np.zeros((n_realizations,dim,dim,1)),             np.zeros((n_realizations,dim,dim,1))

for i in range(n_realizations):
    data0 = loadmat('//dcstorage.lanl.gov/MFR2/misael/h2dataf/{}UHSS_0'.format(i+1))
    poro[i], perm[i] = data0['PORO'], data0['PERMX']
    for j in range(n_timesteps):
        data = loadmat('//dcstorage.lanl.gov/MFR2/misael/h2dataf/{}UHSS_{}'.format(i+1,j))
        pressure[i,j] = data['PRESSURE']
        sat_h2[i,j]   = data['SGAS'] * data['YMF_3']

In [3]:
facies = np.expand_dims(hete.load_facies()[:n_realizations],-1)
facies.shape

Facies shape: (1000, 256, 256)


(5, 256, 256, 1)

In [4]:
t_steps = np.arange(61)
times = np.ones((n_realizations,61,256,256,1))

for i in range(61):
    times[:,i] = times[:,i]*t_steps[i]

print(t_steps.shape, times.shape)

(61,) (5, 61, 256, 256, 1)


In [10]:
X_train = np.concatenate([np.repeat(np.concatenate([poro,perm,facies], -1)[:,np.newaxis,...], n_timesteps, axis=1),times],-1).reshape(n_realizations*n_timesteps,dim,dim,4)
y_train = np.concatenate([sat_h2, pressure],-1).reshape(n_realizations*n_timesteps,dim,dim,2)

X_train = np.moveaxis(X_train, -1, 1)
y_train = np.moveaxis(y_train, -1, 1)

print('X: {} | y: {}'.format(X_train.shape, y_train.shape))

X: (305, 4, 256, 256) | y: (305, 2, 256, 256)


In [7]:
def double_convolution(in_channels, out_channels):
    conv_op = Sequential(
        Conv2d(in_channels, out_channels, kernel_size=3, padding=1),
        ReLU(inplace=True),
        Conv2d(out_channels, out_channels, kernel_size=3, padding=1),
        ReLU(inplace=True))
    return conv_op

class h2_hete_rom(nn.Module):
    def __init__(self):
        super(h2_hete_rom, self).__init__()
        
        self.max_pool2d = nn.MaxPool2d(kernel_size=2, stride=2)
        self.down_convolution_1 = double_convolution(4, 16)
        self.down_convolution_2 = double_convolution(16, 32)
        self.down_convolution_3 = double_convolution(32, 64)
        self.down_convolution_4 = double_convolution(64, 128)
        self.down_convolution_5 = double_convolution(128, 256)

        self.up_transpose_1   = nn.ConvTranspose2d(256, 128, kernel_size=2, stride=2)
        self.up_convolution_1 = double_convolution(256, 128)
        self.up_transpose_2   = nn.ConvTranspose2d(128, 64, kernel_size=2, stride=2)
        self.up_convolution_2 = double_convolution(128, 64)
        self.up_transpose_3   = nn.ConvTranspose2d(64, 32, kernel_size=2, stride=2)
        self.up_convolution_3 = double_convolution(64, 32)
        self.up_transpose_4   = nn.ConvTranspose2d(32, 16, kernel_size=2,  stride=2)
        self.up_convolution_4 = double_convolution(32, 16)

        self.out = nn.Conv2d(16, 2, kernel_size=1) 
        
    def forward(self, x):
        down_1 = self.down_convolution_1(x)
        down_2 = self.max_pool2d(down_1)
        down_3 = self.down_convolution_2(down_2)
        down_4 = self.max_pool2d(down_3)
        down_5 = self.down_convolution_3(down_4)
        down_6 = self.max_pool2d(down_5)
        down_7 = self.down_convolution_4(down_6)
        down_8 = self.max_pool2d(down_7)
        down_9 = self.down_convolution_5(down_8)        
        
        up_1 = self.up_transpose_1(down_9)
        x    = self.up_convolution_1(torch.cat([down_7, up_1], 1))
        up_2 = self.up_transpose_2(x)
        x    = self.up_convolution_2(torch.cat([down_5, up_2], 1))
        up_3 = self.up_transpose_3(x)
        x    = self.up_convolution_3(torch.cat([down_3, up_3], 1))
        up_4 = self.up_transpose_4(x)
        x    = self.up_convolution_4(torch.cat([down_1, up_4], 1))
        out   = self.out(x)
        return out

In [12]:
device = 'cuda'

rom = h2_hete_rom().to(device)
optimizer = NAdam(rom.parameters(), lr=1e-3)
loss_fn = nn.MSELoss()

X_train = torch.Tensor(X_train).to(device)
y_train = torch.Tensor(y_train).to(device)

loss, val_loss = [], []
metrics = {'loss':[], 'val_loss':[]}
epochs = 10
batch_size = 50

for epoch in range(epochs):
    rom.train()
    epoch_loss = 0.0
    for i in range(0, len(X_train), batch_size):
        inp  = X_train[i:i+batch_size]
        true = y_train[i:i+batch_size]
        optimizer.zero_grad()
        pred = rom(inp)
        loss = loss_fn(pred,true)
        loss.backward()
        optimizer.step()
        epoch_loss += loss.item()*inp.size(0)
    metrics['loss'].append(epoch_loss/len(X_train))

OutOfMemoryError: CUDA out of memory. Tried to allocate 154.00 MiB (GPU 0; 4.00 GiB total capacity; 2.70 GiB already allocated; 0 bytes free; 2.89 GiB reserved in total by PyTorch) If reserved memory is >> allocated memory try setting max_split_size_mb to avoid fragmentation.  See documentation for Memory Management and PYTORCH_CUDA_ALLOC_CONF

In [ ]:
X_train.shape

***
# END